In [1]:
from chemblmining import main as miner 
from cnn import main as cnn 
from descriptors import main as desc 

from configparser import ConfigParser 

Using TensorFlow backend.



Please cite HTMD: Doerr et al.(2016)JCTC,12,1845. https://dx.doi.org/10.1021/acs.jctc.6b00049

HTMD Documentation at: https://www.htmd.org/docs/latest/

New devel HTMD version (1.7.34 python[==3.6,==3.5]) is available. You are currently on (1.7.33). Use 'conda update -c acellera htmd' to update to the new version. You might need to update your python version as well if there is no release for your current version.



In [2]:
parser = ConfigParser()
parser.read("config.file")
configPath = parser.get('init', 'configPath')

In [3]:
ligands = miner.main(configPath)

Loading data from file


In [4]:
ligands.shape

(8428, 3)

In [4]:
descriptors, labels = desc.main(configPath, ligands)

Bulding descriptors ...
pdb 12171 not found : 1
0.9966777408637874%
pdb 39897 not found : 2
pdb 45447 not found : 3
2.0052206929283343%
pdb 50610 not found : 4
pdb 54570 not found : 5
pdb 54922 not found : 6
pdb 74706 not found : 7
3.013763644992881%
pdb 76665 not found : 8
pdb 85239 not found : 9
4.022306597057428%
pdb 91225 not found : 10
pdb 92324 not found : 11
pdb 95941 not found : 12
pdb 96195 not found : 13
5.0308495491219745%
6.039392501186521%
pdb 130464 not found : 14
pdb 133399 not found : 15
pdb 153725 not found : 16
7.0479354532510685%
pdb 156482 not found : 17
pdb 164462 not found : 18
8.056478405315614%
pdb 192561 not found : 19
pdb 195415 not found : 20
9.06502135738016%
pdb 206450 not found : 21
pdb 212756 not found : 22
10.073564309444707%
pdb 233022 not found : 23
pdb 233073 not found : 24
pdb 233247 not found : 25
pdb 235612 not found : 26
pdb 249189 not found : 27
11.082107261509256%
pdb 265867 not found : 28
12.090650213573802%
pdb 285792 not found : 29
pdb 287355

In [5]:
descriptors.shape


(8236, 12, 12, 12, 8)

In [7]:
labels

array(['0', '0', '0', ..., '1', '0', '1'],
      dtype='<U32')

In [8]:
from cnn import network as net
from cnn import dataPreparer as dp
from configparser import ConfigParser
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv3D, MaxPooling3D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras import backend as K

In [9]:
datafolder = parser.get('init', 'datafolder')
boxSize = int(parser.get('init', 'boxSize'))
kernelSize = int(parser.get('init', 'kernelSize'))
nClases = int(parser.get('init', 'nClases'))
nEpoch = int(parser.get('init', 'nEpoch'))
batchSize =int( parser.get('init', 'batchSize'))
nFilters = int(parser.get('init', 'nFilters'))

In [18]:
X_train = np.array(descriptors)
Y_train = np.array(labels)
Y_train = Y_train.reshape((-1, 1))
# (1 channel, 25 rows, 25 cols, 25 of depth)
input_shape = (boxSize, boxSize, boxSize,8)

# Init
model = Sequential()

In [19]:
model.add(Conv3D(nFilters,kernelSize,
                            input_shape=input_shape,
                            activation='relu'))

In [20]:
model.add(Flatten())



In [21]:
model.add(Dense(1, activation='relu'))

In [22]:
model.add(Activation('sigmoid'))

In [23]:
from keras.optimizers import Adam
model.compile(loss='binary_crossentropy',
            optimizer= Adam(), metrics=['accuracy'])

In [ ]:
model.fit(X_train, Y_train, batch_size= batchSize, epochs=nEpoch, verbose=1)

Epoch 1/100


In [17]:
Y_train

array(['0', '0', '0', ..., '1', '0', '1'],
      dtype='<U32')

In [24]:
Y_train

array([['0'],
       ['0'],
       ['0'],
       ..., 
       ['1'],
       ['0'],
       ['1']],
      dtype='<U32')

In [26]:
Y_train.T

array([['0', '0', '0', ..., '1', '0', '1']],
      dtype='<U32')

In [28]:
arr = []
for y in Y_train:
   arr.append(y) 

In [31]:
narr = np.array(arr)

In [39]:
narr2 = narr.reshape(-1,1)


In [33]:
arr = arr.reshape((-1, 1))

AttributeError: 'list' object has no attribute 'reshape'

In [40]:
narr2

array([['0'],
       ['0'],
       ['0'],
       ..., 
       ['1'],
       ['0'],
       ['1']],
      dtype='<U32')

In [42]:
Y_train

array([['0'],
       ['0'],
       ['0'],
       ..., 
       ['1'],
       ['0'],
       ['1']],
      dtype='<U32')

In [1]:




from htmd import *
from htmd.molecule import voxeldescriptors as vd
from htmd.molecule import util as u




Please cite HTMD: Doerr et al.(2016)JCTC,12,1845. https://dx.doi.org/10.1021/acs.jctc.6b00049

HTMD Documentation at: https://www.htmd.org/docs/latest/

New devel HTMD version (1.7.35 python[==3.5,==3.6]) is available. You are currently on (1.7.33). Use 'conda update -c acellera htmd' to update to the new version. You might need to update your python version as well if there is no release for your current version.



In [2]:
mol = Molecule("/home/david/Documents/projects/binding-prediction-DL/data/auxQT/8145.pdbqt")

In [3]:
dist = u.maxDistance(mol, sel='all', origin=[0, 0, 0])

In [4]:
dist

6.7099215044251777

In [5]:
point = np.squeeze(mol.coords).mean(axis=0)

In [21]:
desc = vd.getPointDescriptors(mol, point, size=[int(14)]*3)

In [22]:
desc.shape

(14, 14, 14, 8)

In [20]:
desc2 = vd.getPointDescriptors(mol, point, size=[int(7)]*3)

In [31]:
desc2[3][3][3]

array([ 1.        ,  0.99999782,  0.03728058,  0.        ,  1.        ,
        1.        ,  0.        ,  1.        ])

In [32]:
desc[7][7][7]

array([ 1.        ,  0.89496171,  0.00331686,  0.        ,  1.        ,
        0.99988408,  0.        ,  1.        ])

In [11]:
mol2 = Molecule("/home/david/Documents/projects/binding-prediction-DL/data/auxQT/1970.pdbqt")

In [12]:
dist2 = u.maxDistance(mol2, sel='all', origin=[0, 0, 0])

In [13]:
dist2

8.2593653632781194

In [15]:
from htmd.molecule import util as u


In [16]:
u.boundingBox(mol, sel='all')

array([[-4.43400002, -4.3920002 , -1.56900001],
       [ 5.66099977,  6.67700005,  2.7019999 ]], dtype=float32)

In [17]:
from chemblmining import main as miner 
from cnn import main as cnn 
from descriptors import main as desc 

from configparser import ConfigParser


parser = ConfigParser()
parser.read("config.file")
configPath = parser.get('init', 'configPath')
ligands = miner.main(configPath)


Using TensorFlow backend.


Loading data from file


In [19]:
type(ligands[1][2])

numpy.str_